# Analysis of RiboTag RNA-seq data

## Loading packages
Required packages for downstream analysis

In [1]:
suppressMessages(library('DESeq2'))
suppressMessages(library('ggplot2'))
suppressMessages(library('pheatmap'))
suppressMessages(library('RColorBrewer'))
suppressMessages(library('vidger'))
suppressMessages(library('EnhancedVolcano'))
suppressMessages(library('gplots'))
suppressMessages(library('org.Mm.eg.db'))
suppressMessages(library('clusterProfiler'))
suppressMessages(library('sigaR'))
suppressMessages(library('VennDiagram'))
suppressMessages(library('sva'))

options(repr.matrix.max.rows=600, repr.matrix.max.cols=200)
options(repr.plot.width=10, repr.plot.height=10)

Warning message:
“package ‘RColorBrewer’ was built under R version 4.0.5”
Warning message:
“package ‘mgcv’ was built under R version 4.0.5”
Warning message:
“package ‘nlme’ was built under R version 4.0.5”


Set whether to produce plot, set to FALSE for test runs.

In [2]:
bool_plot = TRUE

## Chow data

### Load data

In [3]:
raw <- read.table(file='counts.24sept2019.tsv', header = T, sep='\t')
dat <- raw[,-c(1,12)]
rownames(dat) <- raw[,1]
colnames(dat) <- c("inputCCK1","inputCCK2","inputCCK3","inputCCK4","inputVeh1","inputVeh2","inputVeh3","inputVeh4",
                   "IPcckChow1","IPcckChow2","IPcckChow4","IPvehChow1","IPvehChow2","IPvehChow3","IPvehChow4")

In [4]:
id <- colnames(dat)
condition <- c("Input_CCK","Input_CCK","Input_CCK","Input_CCK","Input_Veh","Input_Veh","Input_Veh","Input_Veh",
               "IPcckChow","IPcckChow","IPcckChow","IPvehChow","IPvehChow","IPvehChow","IPvehChow")
tissue <- c(rep("Input",8),rep("IP",7))
metaData <- data.frame(id, tissue, condition)

### Prepocessing and creating DESeq object

In [5]:
dds <- DESeqDataSetFromMatrix(countData = dat,
                              colData = metaData,
                              design =~ condition
)
featureData <- data.frame(gene=rownames(dat))
mcols(dds) <- DataFrame(mcols(dds), featureData)

dds <- estimateSizeFactors(dds)
idx <- rowMeans(counts(dds, normalized=TRUE)) >= 10
dds <- dds[idx,]
vsd <- vst(dds, 
           fitType="local"
)

vst.data <- varianceStabilizingTransformation(dds, 
                                              blind=TRUE, 
                                              fitType="local"
)

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”


### Visualisations

In [6]:
if(bool_plot==FALSE){ 
    pcaData <- plotPCA(vsd, intgroup=c("condition"), returnData=TRUE)
    percentVar <- round(100 * attr(pcaData, "percentVar"))
    p2 <- ggplot(pcaData, aes(PC1, PC2, color=condition)) +
        geom_point(size=3) +
        geom_label(aes(label=colnames(dat))) +
        xlab(paste0("PC1: ",percentVar[1],"% variance")) + 
        ylab(paste0("PC2: ",percentVar[2],"% variance")) +
        ggtitle("PCA Plot - Chow Data (without ipCCK3)") +
        theme(legend.position="right", 
              legend.text=element_text(size = 10)) + 
        guides(col=guide_legend(nrow=4))
    p2 + ylim(-15,15) + xlim(-40,60)
}

In [7]:
if(bool_plot==FALSE){ 
    ### Visualize the sample distances (euclidian distance) in the heatmaps using pheatmap package
    sampleDistMatrix <- as.matrix(dist(t(assay(vst.data))))
    colors <- colorRampPalette( rev(brewer.pal(9, "Spectral")) )(255)

    # Data frame with column annotations.
    col_ann <- data.frame(condition = colData(dds)$condition)
    rownames(col_ann) <- colnames(sampleDistMatrix)

    # List with colors for each annotation.
    ann_colors <- list(condition = brewer.pal(4, "Set3"))
    names(ann_colors$condition) <- unique(colData(dds)$condition)

    pheatmap(sampleDistMatrix,
             clustering_distance_rows=dist(t(assay(vst.data))),
             clustering_distance_cols=dist(t(assay(vst.data))),
             show_colnames=TRUE,
             show_rownames=FALSE,
             annotation=col_ann,
             annotation_colors=ann_colors,
             col=colors
    )
}

In [8]:
if(bool_plot==FALSE){ 
    # Density plot of the samples from different locations
    p <-assay(vst.data)
    #Sample data
    l <- as.numeric(p[,1:8])
    k <- as.numeric(p[,9:15])
    datt <- data.frame(dens = c(l, k), 
    lines = c(rep("Input",length(l)), rep("IP",length(k))))
    #Plot.
    ggplot(datt, aes(x = dens, fill = lines)) + geom_density(alpha = 0.5)
}

### Differential gene expression analysis

In [9]:
dds <- DESeq(dds, 
             fit='local'
)

using pre-existing size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



In [10]:
res_Input <- results(dds, 
                     contrast=c("condition","Input_CCK","Input_Veh")
)
table(res_Input$padj < 0.05)
res = res_Input[order(res_Input$padj),]


FALSE  TRUE 
16950     3 

In [11]:
results1 = as.data.frame(dplyr::mutate(as.data.frame(res), 
                                       sig=ifelse(res$padj<0.05, "FDR<0.05", "Not Sig")),
                         row.names=rownames(res)
)
results1 = results1[(!is.na(results1$pvalue)),]

In [12]:
# save the results
setwd('/Users/viktorian.miok/Documents/consultation/Tim/Oxytocin_data/results')
write.csv(as.data.frame(results1), file="DifferentialExpressionAnalysis_InputCCK_vs_InputVeh.csv")

In [13]:
if(bool_plot==FALSE){ 
    p <- ggplot2::ggplot(results1, ggplot2::aes(log2FoldChange, -log10(pvalue))) +
         ggplot2::geom_point(ggplot2::aes(col=sig)) + 
         ggplot2::scale_color_manual(values=c("red", "black")) + guides(col = guide_legend(nrow=2)) + 
         ggplot2::ggtitle("Input_CCK_vs_Input_Veh")+xlim(-5,5)

    p + ggrepel::geom_text_repel(data=results1[1:3, ], ggplot2::aes(label=rownames(results1[1:3, ])))
}

In [14]:
res_IP <- results(dds, 
                  contrast=c("condition","IPcckChow","IPvehChow")
)
table(res_IP$padj < 0.05)
res = res_IP[order(res_IP$padj),]


FALSE  TRUE 
16795   158 

In [15]:
results2 = as.data.frame(dplyr::mutate(as.data.frame(res),
                                       sig=ifelse(res$padj<0.05, "FDR<0.05", "Not Sig")),
                        row.names=rownames(res)
)
results2 = results2[(!is.na(results2$pvalue)),]

In [16]:
write.csv(as.data.frame(results2), file="DifferentialExpressionAnalysis_IPcck_vs_IPveh.csv")

In [17]:
if(bool_plot==FALSE){ 
    p <-  ggplot2::ggplot(results2, ggplot2::aes(log2FoldChange, -log10(padj))) +
          ggplot2::geom_point(ggplot2::aes(col=results2$sig)) + 
          ggplot2::scale_color_manual(values=c("red", "black")) + guides(col = guide_legend(nrow=2)) + 
          ggplot2::ggtitle("IPcckChow_vs_IPvehChow")

    p + ggrepel::geom_text_repel(data=results2[1:20,], ggplot2::aes(label=rownames(results2[1:20,])))
}

In [18]:
dist.pear <- function(x) as.dist(1-cor(t(x)))
hclust.ave <- function(x) hclust(x, method="average")

In [19]:
if(bool_plot==FALSE){ 
    p = assay(vst.data)
    sigChow <- rownames(res[which(res$padj < 0.05),])
    diet = p[rownames(p)%in%sigChow,]
    ord <- heatmap.2(diet[,9:15],
                     col=colors,
                     scale="row",
                     trace="none",
                     main="Signif. Gen: IPcckChow_vs_IPvehChow",
                     Rowv=TRUE,
                     Colv=FALSE,
                     cexCol=2,
                     margins=c(15, 33),
                     keysize=1,
                     distfun=dist.pear,
                     hclustfun=hclust.ave
    )
}

In [20]:
resIP1 <- res_IP[which(res_IP[,6]<0.05),]

In [21]:
sig.gene <- bitr(rownames(resIP1), 
                 fromType="SYMBOL",
                 toType="ENTREZID",
                 OrgDb=org.Mm.eg.db
)
ego1 <- enrichGO(gene=sig.gene[,2], 
                 OrgDb=org.Mm.eg.db,
                 ont="BP", 
                 pAdjustMethod="BH",
                 pvalueCutoff=0.5, 
                 readable=TRUE
)

'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(rownames(resIP1), fromType = "SYMBOL", toType = "ENTREZID", :
“17.09% of input gene IDs are fail to map...”


In [22]:
df_total <- ego1@result[1:20,]

In [23]:
options(repr.plot.width=20, repr.plot.height=16)

In [24]:
if(bool_plot==FALSE){
    ggplot(df_total, aes(Description, Count)) +
           geom_bar(stat = "identity", aes(fill = pvalue), position = position_stack(reverse = TRUE)) +
           coord_flip() + 
           scale_color_gradient(low="red", high="yellow") +
           theme(axis.title=element_text(size=20,face="bold"),
                 axis.text = element_text(size = 25),
                 title=element_text(size=20,face="bold"),
                 legend.text=element_text(size=20)) + 
           ylab("Nr. of Genes in Pathway") + 
           xlab("Pathway Names") +
           ggtitle("GO Significant Pathways")
}

## HFD data

### Load data

In [25]:
setwd('/Users/viktorian.miok/Documents/consultation/Tim/Oxytocin_data')
raw_hfd <- read.table(file='counts.feb2020.tsv', header = T, sep='\t')
MUC14082 <- read.table(file='MUC14082.counts', header = F, sep='\t')
setwd('/Users/viktorian.miok/Documents/consultation/Tim/Oxytocin_data/results')

In [26]:
raw_hfd <- cbind(raw_hfd,MUC14082)
rownames(raw_hfd) <- raw_hfd[,1]
raw_hfd <- raw_hfd[,c(2,14,3:12)]

In [27]:
id <- colnames(raw_hfd) <- c("IPvehHfd1","IPvehHfd2","IPvehHfd3","IPvehHfd4",
                             "IPcckHfd1","IPcckHfd2","IPcckHfd3","IPcckHfd4",
                             "IPgluc1","IPgluc2","IPgluc3","IPgluc4")

condition <- c("IPvehHfd","IPvehHfd","IPvehHfd","IPvehHfd",
               "IPcckHfd","IPcckHfd","IPcckHfd","IPcckHfd",
               "IPgluc","IPgluc","IPgluc","IPgluc")
metaData <- data.frame(id, condition)

### Prepocessing and creating DESeq object

In [28]:
dds <- DESeqDataSetFromMatrix(countData=raw_hfd,
                              colData=metaData, 
                              design =~ condition
)
featureData <- data.frame(gene=rownames(dat))
mcols(dds) <- DataFrame(mcols(dds), featureData)

dds <- estimateSizeFactors(dds)
idx <- rowMeans(counts(dds, normalized=TRUE)) >= 10
dds <- dds[idx,]
vsd <- vst(dds, 
           fitType="local"
)

vst.data <- varianceStabilizingTransformation(dds, 
                                              blind=TRUE, 
                                              fitType="local"
)

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”


### Visualisations

In [29]:
if(bool_plot==FALSE){
    pcaData <- plotPCA(vsd, intgroup=c("condition"), returnData=TRUE)
    percentVar <- round(100 * attr(pcaData, "percentVar"))
    p2 <- ggplot(pcaData, aes(PC1, PC2, color = condition)) +
                geom_point(size=3)+
                geom_label(aes(label = colnames(raw_hfd))) +
                xlab(paste0("PC1: ",percentVar[1],"% variance")) + 
                ylab(paste0("PC2: ",percentVar[2],"% variance")) +
                ggtitle("PCA Plot - All Samples HFD data") +
                theme(legend.position = "right", legend.text = element_text(size = 10)) + 
                guides(col = guide_legend(nrow=4))
    p2 + xlim(-15,25) 
}

In [30]:
if(bool_plot==FALSE){
    ### Visualize the sample distances (euclidian distance) in the heatmaps using pheatmap package
    sampleDistMatrix <- as.matrix(dist(t(assay(vst.data))))
    colors <- colorRampPalette( rev(brewer.pal(9, "Spectral")) )(255)

    # Data frame with column annotations.
    col_ann <- data.frame(condition = colData(dds)$condition)
    rownames(col_ann) <- colnames(sampleDistMatrix)

    # List with colors for each annotation.
    ann_colors <- list(condition = brewer.pal(3, "Set3"))
    names(ann_colors$condition) <- unique(colData(dds)$condition)

    pheatmap(sampleDistMatrix,
             clustering_distance_rows=dist(t(assay(vst.data))),
             clustering_distance_cols=dist(t(assay(vst.data))),
             show_colnames=TRUE,
             show_rownames=FALSE,
             annotation=col_ann,
             annotation_colors=ann_colors,
             col=colors
    )
}

In [31]:
dds <- DESeq(dds, 
             fit='local'
)

using pre-existing size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



In [32]:
hdfcom <- results(dds, contrast = c("condition","IPcckHfd","IPvehHfd"))
table(hdfcom$padj < 0.05)
res = hdfcom[order(hdfcom$padj),]


FALSE  TRUE 
15554     4 

In [33]:
results3 = as.data.frame(dplyr::mutate(as.data.frame(res), 
                                       sig=ifelse(res$padj<0.05, "FDR<0.05", "Not Sig")),
                         row.names=rownames(res)
)
results3 = results3[(!is.na(results3$pvalue)),]

In [34]:
write.csv(as.data.frame(results3), file="DifferentialExpressionAnalysis_IPcckHfd_vs_IPvehHfd.csv")

In [35]:
if(bool_plot==FALSE){
    p = ggplot2::ggplot(results3, ggplot2::aes(log2FoldChange, -log10(padj))) +
        ggplot2::geom_point(ggplot2::aes(col=sig)) + 
        ggplot2::scale_color_manual(values=c("red", "black")) +
        guides(col=guide_legend(nrow=2)) +
        ggplot2::ggtitle("IPcckHfd_vs_IPvehHfd") + 
        xlim(-5,5)

    p + ggrepel::geom_text_repel(data=results3[1:5,], ggplot2::aes(label=rownames(results3[1:5,])))
}

In [36]:
IPvehChow_vs_IPvehChow = rownames(results2[which(results2$sig == "FDR<0.05"),]) 
IPcckHfd_vs_IPvehHfd = rownames(results3[which(results3$sig == "FDR<0.05"),])  

In [37]:
if(bool_plot==FALSE){
    vp <- venn.diagram(list(IPvehChow_vs_IPvehChow=IPvehChow_vs_IPvehChow, 
                            IPcckHfd_vs_IPvehHfd=IPcckHfd_vs_IPvehHfd),
                       fill=c("red", "olivedrab3"),
                       filename=NULL,
                       cex=1.5, 
                       col="transparent",
                       cat.cex=1.3,
                       margin=0.25,
                       main.cex=2);
    grid.draw(vp) 
}

In [38]:
if(bool_plot==FALSE){
    p = assay(vst.data)

    diet = p[rownames(p)%in%sigChow,]
    heatmap.2(diet[,c(5:8,1:4)],
              col=colors,scale="row",
              trace="none",
              main="Significant Chow Genes in IPcckHfd_and_IPvehHfd",
              Rowv=TRUE,
              Colv=FALSE,
              cexCol=2,
              margins=c(15,33),
              keysize=1,
              distfun=dist.pear,
              hclustfun=hclust.ave
    )
}

## All samples together

### Load data

In [39]:
setwd('/Users/viktorian.miok/Documents/consultation/Tim/Oxytocin_data')
raw_hfd <- read.table(file='counts.feb2020.tsv', header = T, sep='\t')
MUC14082 <- read.table(file='MUC14082.counts', header = F, sep='\t')
raw_chow <- read.table(file='counts.24sept2019.tsv', header = T, sep='\t')
setwd('/Users/viktorian.miok/Documents/consultation/Tim/Oxytocin_data/results')

In [40]:
rownames(raw_chow) <- raw_chow[,1]
raw_chow <- raw_chow[,-c(1,12)]

In [41]:
raw_hfd <- cbind(raw_hfd,MUC14082)
rownames(raw_hfd) <- raw_hfd[,1]
raw_hfd <- raw_hfd[,c(2,14,3:12)]

In [42]:
dat <- cbind(raw_chow,raw_hfd)

In [43]:
id <- colnames(dat) <- c("inputCCK1","inputCCK2","inputCCK3","inputCCK4",
                         "inputVeh1","inputVeh2","inputVeh3","inputVeh4",
                         "IPcckChow1","IPcckChow2","IPcckChow3",
                         "IPvehChow1","IPvehChow2","IPvehChow3","IPvehChow4",
                         "IPvehHfd1","IPvehHfd2","IPvehHfd3","IPvehHfd4",
                         "IPcckHfd1","IPcckHfd2","IPcckHfd3","IPcckHfd4",
                         "IPgluc1","IPgluc2","IPgluc3","IPgluc4")
batch <- as.factor(c(rep(1,ncol(raw_chow)),rep(2,ncol(raw_hfd))))
condition <- c("InputCCK","InputCCK","InputCCK","InputCCK",
               "InputVeh","InputVeh","InputVeh","InputVeh",
               "IPcckChow","IPcckChow","IPcckChow",
               "IPvehChow","IPvehChow","IPvehChow","IPvehChow",
               "IPvehHfd","IPvehHfd","IPvehHfd","IPvehHfd",
               "IPcckHfd","IPcckHfd","IPcckHfd","IPcckHfd",
               "IPgluc","IPgluc","IPgluc","IPgluc")
metaData <- data.frame(id, batch, condition)

### Prepocessing and creating DESeq object

In [44]:
dat <- ComBat_seq(as.matrix(dat),
                  batch=batch,
                  group=NULL
)

Found 2 batches
Using null model in ComBat-seq.
Adjusting for 0 covariate(s) or covariate level(s)
Estimating dispersions
Fitting the GLM model
Shrinkage off - using GLM estimates for parameters
Adjusting the data


In [45]:
dds <- DESeqDataSetFromMatrix(countData=dat,
                              colData=metaData,
                              design =~ condition
)
dds <- estimateSizeFactors(dds)
idx <- rowMeans(counts(dds, normalized=TRUE)) >= 10
dds <- dds[idx,]
vsd <- vst(dds, fitType="local")

vst.data <- varianceStabilizingTransformation(dds, blind = TRUE, fitType = "local")

converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”


### Visualisation

In [46]:
if(bool_plot==FALSE){
    pcaData <- plotPCA(vsd, intgroup=c("condition"), returnData=TRUE)
    percentVar <- round(100 * attr(pcaData, "percentVar"))
    p2 <- ggplot(pcaData, aes(PC1, PC2, color = condition)) +
                geom_point(size=3) +
                geom_label(aes(label = colnames(dat))) +
                xlab(paste0("PC1: ",percentVar[1],"% variance")) + 
                ylab(paste0("PC2: ",percentVar[2],"% variance")) +
                ggtitle("PCA Plot - All Samples") + # Batch Corrected
                theme(legend.position = "right", legend.text = element_text(size = 15)) + 
                guides(col = guide_legend(nrow=4))
    p2 + theme(axis.title=element_text(size=20,face="bold"),
               axis.text=element_text(size=20),
              title=element_text(size=20,face="bold")) #+ xlim(-80,50) #+ ylim(-20,20) + xlim(-15,25)
}

In [47]:
if(bool_plot==FALSE){
    ### Visualize the sample distances (euclidian distance) in the heatmaps using pheatmap package
    sampleDistMatrix <- as.matrix(dist(t(assay(vst.data))))
    colors <- colorRampPalette( rev(brewer.pal(9, "Spectral")) )(255)

    # Data frame with column annotations.
    col_ann <- data.frame(condition = colData(dds)$condition)
    rownames(col_ann) <- colnames(sampleDistMatrix)

    # List with colors for each annotation.
    ann_colors <- list(condition = brewer.pal(7, "Set3"))
    names(ann_colors$condition) <- unique(colData(dds)$condition)
    #ann_colors[[1]] <- ann_colors$condition[-3]

    pheatmap(sampleDistMatrix,
             clustering_distance_rows=dist(t(assay(vst.data))),
             clustering_distance_cols=dist(t(assay(vst.data))),
             show_colnames=TRUE,
             show_rownames=FALSE,
             annotation=col_ann,
             annotation_colors=ann_colors,
             col=colors)
}

### Diferential gene expression

In [48]:
dds <- DESeq(dds, 
             fit='local'
)

using pre-existing size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



## IP_CCK-HFD vs. IP_Veh-Chow  

In [49]:
vehChowVScckHfd <- results(dds, 
                           contrast=c("condition","IPcckHfd","IPvehChow")
)
table(vehChowVScckHfd$padj < 0.05)
res = vehChowVScckHfd[order(vehChowVScckHfd$padj),]


FALSE  TRUE 
13145  2408 

In [50]:
results4 = as.data.frame(dplyr::mutate(as.data.frame(res), 
                                       sig=ifelse(res$padj<0.05, "FDR<0.05", "Not Sig")),
                         row.names=rownames(res)
)
results4 = results4[(!is.na(results4$pvalue)),]
results4 = results4[(!is.na(results4$padj)),]

In [51]:
write.csv(as.data.frame(results4), file="DifferentialExpressionAnalysis_IPcckHfd_vs_IPvehChow.csv")

In [52]:
if(bool_plot==FALSE){
    p = ggplot2::ggplot(results4, ggplot2::aes(log2FoldChange, -log10(padj))) +
        ggplot2::geom_point(ggplot2::aes(col = sig)) + 
        ggplot2::scale_color_manual(values = c("red", "black")) +
              guides(col = guide_legend(nrow=2)) +
        ggplot2::ggtitle("IPcckHFD_vs_IPVehChow") + 
        ggrepel::geom_text_repel(data=results4[1:20, ], ggplot2::aes(label=rownames(results4[1:20, ])))
    p + theme(axis.title=element_text(size=20,face="bold"), axis.text = element_text(size = 20),
              title=element_text(size=20, face="bold"))
}

In [53]:
if(bool_plot==FALSE){
    p = assay(vst.data)
    diet = p[rownames(p)%in%rownames(res[which(res$padj<0.05),]),]
    heatmap.2(diet[,c(12:15,20:23)], 
              col=colors,scale="row",
              trace="none",
              main="Signif. Gen:IP_CCK-HFD_vs_IP_Veh-Chow",
              Rowv=TRUE,
              Colv=FALSE,
              cexCol=2,
              margins=c(15, 33),
              keysize=1,
              distfun=dist.pear,
              hclustfun=hclust.ave
    )
}

In [54]:
resCP1 <- res[which(res[,6]<0.05),]

In [55]:
sig.gene <- bitr(rownames(resCP1),
                 fromType="SYMBOL",
                 toType="ENTREZID",
                 OrgDb=org.Mm.eg.db
)
ego1 <- enrichGO(gene=sig.gene[,2],
                 OrgDb=org.Mm.eg.db, 
                 ont="BP", 
                 pAdjustMethod="BH",
                 pvalueCutoff=0.5, 
                 readable=TRUE
)

'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(rownames(resCP1), fromType = "SYMBOL", toType = "ENTREZID", :
“13.66% of input gene IDs are fail to map...”


In [56]:
ego1Path <- data.frame(ego1@result$Description, 
                       ego1@result$p.adjust,
                       ego1@result$geneID,
                       ego1@result$Count
)

In [57]:
ego1Path=ego1Path[which(ego1Path[,2]<0.05),]

In [58]:
colnames(ego1Path) <- c("PathwayName","Pathway_AdjP-val","GeneNanme","GeneCount")

In [59]:
write.csv(ego1Path,"GO_Signif_Pathways-IPcckHfd_vs_IPvehChow.csv")

In [60]:
df_total <- data.frame(ego1@result$Description[1:20], 
                       ego1@result$p.adjust[1:20],
                       ego1@result$Count[1:20]
)

colnames(df_total) <- c("pathway","padjPath","count")

In [61]:
options(repr.plot.width=20, repr.plot.height=16)

In [62]:
if(bool_plot==FALSE){
    ggplot(df_total, aes(pathway, count)) + 
        geom_bar(stat = "identity", aes(fill = padjPath), position = position_stack(reverse = TRUE)) + 
        coord_flip() +
        scale_color_gradient(low="red", high="yellow")+
        theme(axis.title=element_text(size=20,face="bold"),
              axis.text = element_text(size = 20),
              title=element_text(size=20,face="bold")) +
        ylab("Nr. of Genes in Pathway") + 
        xlab("Pathway Names") + 
        ggtitle("GO Significant Pathways")
}

In [63]:
df_total = data.frame()
for(i in 1:20){
    p <- as.data.frame(resCP1[which(rownames(resCP1)%in%unlist(strsplit(ego1@result$geneID[i],"/"))),c(2,6)])
    p$pathway <- ego1@result$Description[i]
    p$padjPath <- ego1@result$p.adjust[i]
    p$count <- ego1@result$Count[i]
    df_total <- rbind(df_total,p)
}
df_total[is.na(df_total[,2]),2] <- 0.05

In [64]:
if(bool_plot==FALSE){
    ggplot(df_total, aes(log2FoldChange, pathway)) + 
        scale_color_gradient(low="red", high="yellow") +
        geom_point(aes(color = -log10(padj))) +
        geom_vline(xintercept = 0) +
        theme(axis.title=element_text(size=20,face="bold"),
              axis.text=element_text(size = 20),
              title=element_text(size=20,face="bold")) +
        ylab("Pathway Names") + 
        ggtitle("Significant Genes")
}

## IP_Veh-HFD vs. IP_Veh-Chow 

In [65]:
vehChowVSvehHfd <- results(dds, contrast = c("condition","IPvehHfd","IPvehChow"))
table(vehChowVSvehHfd$padj < 0.05)
res = vehChowVSvehHfd[order(vehChowVSvehHfd$padj),]


FALSE  TRUE 
13065  3127 

In [66]:
results5 = as.data.frame(dplyr::mutate(as.data.frame(res), 
                                       sig=ifelse(res$padj<0.05, "FDR<0.05", "Not Sig")),
                         row.names=rownames(res)
)
results5 = results5[(!is.na(results5$pvalue)),]
results5 = results5[(!is.na(results5$padj)),]

In [67]:
write.csv(as.data.frame(results5), file="DifferentialExpressionAnalysis_IPvehHfd_vs_IPvehChow.csv")

In [68]:
if(bool_plot==FALSE){
    p = ggplot2::ggplot(results5, ggplot2::aes(log2FoldChange, -log10(padj))) +
        ggplot2::geom_point(ggplot2::aes(col = sig)) + 
        ggplot2::scale_color_manual(values = c("red", "black")) +
        guides(col = guide_legend(nrow=2)) + 
        ggplot2::ggtitle("IPVehHFD_vs_IP_VehChow")+
        ggrepel::geom_text_repel(data=results5[1:20, ], ggplot2::aes(label=rownames(results5[1:20, ])))

    p + theme(axis.title=element_text(size=20,face="bold"),
              axis.text=element_text(size = 20),
              title=element_text(size=20,face="bold"))
}

In [69]:
if(bool_plot==FALSE){
    p = assay(vst.data)
    diet = p[rownames(p)%in%rownames(res[which(res$padj<0.05),]),]
    heatmap.2(diet[,c(12:15,16:19)], 
              col=colors,
              scale="row",
              trace="none",
              main="Signif.Gen:IPvehHfd_vs_IPvehChow",
              Rowv=TRUE,
              Colv=FALSE,
              cexCol=2,
              margins=c(15, 33),
              keysize=1,
              distfun=dist.pear,
              hclustfun=hclust.ave
    )
}

In [70]:
resCP2 <- res[which(res[,6]<0.05),]

In [71]:
sig.gene <- bitr(rownames(resCP2),
                 fromType="SYMBOL",
                 toType="ENTREZID",
                 OrgDb=org.Mm.eg.db
)
ego2 <- enrichGO(gene=sig.gene[,2], 
                 OrgDb=org.Mm.eg.db,
                 ont="BP", 
                 pAdjustMethod="BH",
                 pvalueCutoff=0.5, 
                 readable=TRUE
)

'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(rownames(resCP2), fromType = "SYMBOL", toType = "ENTREZID", :
“17.01% of input gene IDs are fail to map...”


In [72]:
ego2Path <- data.frame(ego2@result$Description, 
                       ego2@result$p.adjust, 
                       ego2@result$geneID, 
                       ego2@result$Count
)

In [73]:
ego2Path=ego2Path[which(ego2Path[,2]<0.05),]

In [74]:
colnames(ego2Path) <- c("PathwayName","Pathway_AdjP-val","GeneNanme","GeneCount")

In [75]:
write.csv(ego2Path,"GO_Signif_Pathways-IPvehHfd_vs_IPvehChow.csv")

In [76]:
df_total <- data.frame(ego2@result$Description[1:20], 
                       ego2@result$p.adjust[1:20], 
                       ego2@result$Count[1:20]
)

colnames(df_total) <- c("pathway","padjPath","count")

In [77]:
if(bool_plot==FALSE){
    ggplot(df_total, aes(pathway, count)) +
          geom_bar(stat = "identity", aes(fill = padjPath), 
                   position = position_stack(reverse = TRUE)) +
          coord_flip() +
          scale_color_gradient(low="red", high="yellow") +
          theme(axis.title=element_text(size=20,face="bold"),
                axis.text = element_text(size = 20),
                title=element_text(size=20,face="bold")) +
          ylab("Nr. of Genes in Pathway") +
          xlab("Pathway Names") + 
          ggtitle("GO Significant Pathways")
}

In [78]:
df_total = data.frame()
for(i in 1:20){
    p <- as.data.frame(resCP2[which(rownames(resCP2)%in%unlist(strsplit(ego2@result$geneID[i],"/"))),c(2,6)])
    p$pathway <- ego2@result$Description[i]
    p$padjPath <- ego2@result$p.adjust[i]
    p$count <- ego2@result$Count[i]
    df_total <- rbind(df_total,p)
}
df_total[is.na(df_total[,2]),2] <- 0.05

In [79]:
if(bool_plot==FALSE){
    ggplot(df_total, aes(log2FoldChange, pathway)) + 
         scale_color_gradient(low="red", high="yellow") +
         geom_point(aes(color = -log10(padj))) + 
         geom_vline(xintercept = 0)+
         theme(axis.title=element_text(size=20,face="bold"),
               axis.text = element_text(size = 20),
               title=element_text(size=20,face="bold")) +
         ylab("Pathway Names") + 
         ggtitle("Significant Genes")
}

## IP_Glucose vs. IP_Veh-Chow 

In [80]:
vehHfdVScckHfd <- results(dds, contrast = c("condition","IPgluc","IPvehChow"))
table(vehHfdVScckHfd$padj < 0.05)
res = vehHfdVScckHfd[order(vehHfdVScckHfd$padj),]


FALSE  TRUE 
15505  1008 

In [81]:
results6 = as.data.frame(dplyr::mutate(as.data.frame(res),
                                       sig=ifelse(res$padj<0.05, "FDR<0.05", "Not Sig")), 
                         row.names=rownames(res)
)
results6 = results6[(!is.na(results6$pvalue)),]

In [82]:
write.csv(as.data.frame(results6), file="DifferentialExpressionAnalysis_IPgluc_vs_IPvehChow.csv")

In [83]:
if(bool_plot==FALSE){
    p <-  ggplot2::ggplot(results6, ggplot2::aes(log2FoldChange, -log10(padj))) +
          ggplot2::geom_point(ggplot2::aes(col = sig)) +
          ggplot2::scale_color_manual(values = c("red", "black")) +
          guides(col = guide_legend(nrow=2)) + 
          ggplot2::ggtitle("IP_Glucose_vs_IP_Veh-Chow")+#xlim(-10,10)+ylim(0,40) + 
          ggrepel::geom_text_repel(data=results6[1:20, ], ggplot2::aes(label=rownames(results6[1:20, ])))

    p + theme(axis.title=element_text(size=20,face="bold"),
              axis.text = element_text(size = 20),
              title=element_text(size=20,face="bold"))
}

In [84]:
if(bool_plot==FALSE){
    p = assay(vst.data)
    diet <- p[rownames(p)%in%rownames(res[which(res$padj<0.05),]),]
    heatmap.2(diet[,c(12:15,24:27)], 
              col=colors,scale="row",
              trace="none",
              main="Top Genes - IP_Glucose vs. IP_Veh-Chow",
              Rowv=TRUE,
              Colv=FALSE,
              cexCol=2,
              margins=c(15, 33),
              keysize=1,
              distfun=dist.pear,
              hclustfun=hclust.ave
    )
}

In [85]:
resCP3 <- res[which(res[,6]<0.05),]

In [86]:
sig.gene <- bitr(rownames(resCP3), 
                 fromType="SYMBOL",
                 toType="ENTREZID",
                 OrgDb=org.Mm.eg.db
)
ego3 <- enrichGO(gene=sig.gene[,2], 
                 OrgDb=org.Mm.eg.db, 
                 ont="BP",
                 pAdjustMethod="BH",
                 pvalueCutoff=0.5, 
                 readable=TRUE
)

'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(rownames(resCP3), fromType = "SYMBOL", toType = "ENTREZID", :
“9.72% of input gene IDs are fail to map...”


In [87]:
ego3Path <- data.frame(ego3@result$Description, 
                       ego3@result$p.adjust,
                       ego3@result$geneID, 
                       ego3@result$Count
)

In [88]:
ego3Path=ego3Path[which(ego3Path[,2]<0.05),]

In [89]:
colnames(ego3Path) <- c("PathwayName","Pathway_AdjP-val","GeneNanme","GeneCount")

In [90]:
write.csv(ego3Path,"GO_Signif_Pathways-Treatment_Effect_IPgluc_vs_IPvehChow.csv")

In [91]:
df_total <- data.frame(ego3@result$Description[1:20], 
                       ego3@result$p.adjust[1:20],
                       ego3@result$Count[1:20]
)

colnames(df_total) <- c("pathway","padjPath","count")

In [92]:
if(bool_plot==FALSE){
    ggplot(df_total, aes(pathway, count)) + 
        geom_bar(stat = "identity", aes(fill = padjPath), position = position_stack(reverse = TRUE)) + 
        coord_flip() +
        scale_color_gradient(low="red", high="yellow")+
        theme(axis.title=element_text(size=20,face="bold"),
              axis.text = element_text(size = 20),
              title=element_text(size=20,face="bold")) +
        ylab("Nr. of Genes in Pathway") + 
        ggtitle("GO Significant Pathways")
}

In [93]:
df_total = data.frame()
for(i in 1:20){
    p <- as.data.frame(resCP3[which(rownames(resCP3)%in%unlist(strsplit(ego1@result$geneID[i],"/"))),c(2,6)])
    p$pathway <- ego1@result$Description[i]
    p$padjPath <- ego1@result$p.adjust[i]
    p$count <- ego1@result$Count[i]
    df_total <- rbind(df_total,p)
}
df_total[is.na(df_total[,2]),2] <- 0.05

In [94]:
if(bool_plot==FALSE){
    ggplot(df_total, aes(log2FoldChange, pathway)) + 
        scale_color_gradient(low="red", high="yellow") +
        geom_point(aes(color = -log10(padj))) +
        geom_vline(xintercept = 0) +
        theme(axis.title=element_text(size=20,face="bold"),
              axis.text = element_text(size = 20),
              title=element_text(size=20,face="bold")) +
        ylab("Pathway Names")+ggtitle("Significant Genes")
}

## Venn Diagram

In [95]:
IPvehChow_vs_IPcckHfd_down <- rownames(results4[which((results4$sig=="FDR<0.05")&(results4$log2FoldChange<0)),]) 
IPvehChow_vs_IPcckHfd_up <- rownames(results4[which((results4$sig=="FDR<0.05")&(results4$log2FoldChange>0)),]) 
IPvehChow_vs_IPvehHfd_down <- rownames(results5[which((results5$sig=="FDR<0.05")&(results5$log2FoldChange<0)),])
IPvehChow_vs_IPvehHfd_up <- rownames(results5[which((results5$sig=="FDR<0.05")&(results5$log2FoldChange>0)),])
IPvehChow_vs_IPglucose_down <- rownames(results6[which((results6$sig=="FDR<0.05")&(results6$log2FoldChange<0)),])
IPvehChow_vs_IPglucose_up <- rownames(results6[which((results6$sig=="FDR<0.05")&(results6$log2FoldChange>0)),])

In [96]:
if(bool_plot==FALSE){
    vp <- venn.diagram(list(IPvehChow_vs_IPcckHfd_up = IPvehChow_vs_IPcckHfd_up, 
                            IPvehChow_vs_IPvehHfd_up = IPvehChow_vs_IPvehHfd_up, 
                            IPvehChow_vs_IPglucose_up = IPvehChow_vs_IPglucose_up),
                       fill=c("red", "olivedrab3", "#56B4E9"),
                       filename=NULL,
                       cex=1.5, 
                       col="transparent",
                       cat.cex=1.3, 
                       margin=0.15, 
                       main="Chow vs. HFHS Diet - up-regulated", 
                       main.cex=2);
    grid.draw(vp)
}

In [97]:
if(bool_plot==FALSE){
    vp <- venn.diagram(list(IPvehChow_vs_IPcckHfd_down = IPvehChow_vs_IPcckHfd_down, 
                            IPvehChow_vs_IPvehHfd_down = IPvehChow_vs_IPvehHfd_down, 
                            IPvehChow_vs_IPglucose_down = IPvehChow_vs_IPglucose_down),
                       fill=c("red", "olivedrab3", "#56B4E9"),
                       filename=NULL, 
                       cex=1.5, 
                       col="transparent",
                       cat.cex=1.3,
                       margin=0.15, 
                       main="Chow vs. HFHS Diet - down-regulated", 
                       main.cex=2);
    grid.draw(vp)
}